In [0]:
!pip install lifelines

In [0]:
import pandas as pd, numpy as np, re
from numpy import exp, mean
from fancyimpute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from lifelines import CoxPHFitter, AalenJohansenFitter
# import statsmodels.api as sm
# from matplotlib import pyplot as plt
# from resample.bootstrap import bootstrap_ci, bootstrap

In [0]:
from google.colab import drive
drive.mount('/content/Drive')

In [0]:
# whole cohort (N=1,478,506)
d = pd.read_csv('Drive/My Drive/facility/Fortable3mi_ld_dec_censored.csv')
d['rucc_rural'] = np.where(d.RUCC_2013.isin([1, 2, 3]), 0, (np.where(d.RUCC_2013.isna(), np.nan, 1)))
d = d.drop(['RUCC_2013'], axis=1)
d.loc[d['wl_time']>24, 'wl']=0
d.loc[d['wl_time']>24, 'wl_time']=24
d.loc[d['ld_time']>24, 'livingd']=0
d.loc[d['ld_time']>24, 'ld_time']=24
d.loc[d['dec_time']>24, 'deceasedt']=0
d.loc[d['dec_time']>24, 'dec_time']=24
# d.to_csv('drive/My Drive/facility/jama_round4.csv', index=False)

In [0]:
# Random forest imputation for categorical
for var in ['dialysis_mod1', 'insurance_esrd', 'rucc_rural', 'nephcare_cat2']:
    pred = ['sex_new', 'age_cat', 'race_new', var]
    imputer = IterativeImputer(n_iter=1, random_state=7, predictor=RandomForestClassifier(n_estimators=10))
    imputed = pd.DataFrame(imputer.fit_transform(d[pred]), columns=pred)
    d = d.drop(var, axis=1).join(imputed[var])

In [0]:
# Bayesian Ridge linear imputation for continuous
for var in ['Hospitalization_Rate_facility', 'Mortality_Rate_Facility', 'NEAR_DIST']:
    completed = []
    for i in range(5):
        pred = ['sex_new', 'age_cat', 'race_new', var]
        imputer = IterativeImputer(n_iter=5, sample_posterior=True, random_state=i)
        completed.append(imputer.fit_transform(d[pred]))
    completed_mean = np.mean(completed, axis=0)
    imputed = pd.DataFrame(completed_mean, columns=pred)
    if var == 'NEAR_DIST':
        m = imputed[imputed.NEAR_DIST > 0].NEAR_DIST.mean()
        imputed.NEAR_DIST = np.where(imputed.NEAR_DIST < 0, m, imputed.NEAR_DIST)
    d = d.drop(var, axis=1).join(imputed[var])

ideal = d[(d.INC_AGE < 66) &
  (d.pvasc_new == 0) &
  (d.chf == 0) &
  (d.cva_new == 0) &
  (d.PATTXOP_MEDUNFITn == 0)].reset_index(drop=True)

In [0]:
# standard cohort
PH_data = d[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2', 'profit_txc', 'profit_hosp',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data = PH_data.join(pd.get_dummies(PH_data.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.age_cat, [5, 1, 2, 3, 4, 6], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data = PH_data.join(pd.get_dummies(PH_data.race_new, prefix='race_new', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.chain_class2, [6,5,2,1,3,4], True), prefix='chain_class2', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_txc, prefix='profit_txc', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_hosp, prefix='profit_hosp', drop_first=True))
PH_data = PH_data.drop(['dialysis_mod1', 'esrd_cause', 'age_cat', 'race_new', 'chain_class2', 'insurance_esrd', 'network_us_region_dfr', 'profit_txc', 'profit_hosp'], axis=1)

# ideal cohort
PH_data_ideal = ideal[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.age_cat, [5, 1, 2, 3, 4], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.race_new, prefix='race_new', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.chain_class2, [6,5,2,1,3,4], True), prefix='chain_class2', drop_first=True))
PH_data_ideal = PH_data_ideal.drop(['dialysis_mod1', 'esrd_cause', 'age_cat', 'race_new', 'chain_class2', 'insurance_esrd', 'network_us_region_dfr'], axis=1)

**Table 2**

In [0]:
varlist=['age_cat',
 'sex_new',
 'race_new',
 'insurance_esrd',
 'esrd_cause',
 'dialysis_mod1',
 'bmi_35',
 'chf',
 'ashd_new',
 'other_cardiac',
 'cva_new',
 'pvasc_new',
 'hypertension',
 'diabetes',
 'copd_new',
 'smoke_new',
 'cancer_new',
 'nephcare_cat2',
 'PATTXOP_MEDUNFITn',
 'network_us_region_dfr',
 'rucc_rural',
 'NEAR_DIST']


In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in varlist:
    crude = '|'.join([exposure, time, status])
    cph.fit(PH_data.filter(regex=crude), duration_col=time, event_col=status, step_size=0.5)
    print(round(pd.concat([exp(cph.hazards_).rename('HR'), exp(cph.confidence_intervals_)], 1), 2))

In [0]:
for case, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  tmp = d
  count = tmp.groupby('chain_class2')[case].sum().map('{:,}'.format)
  pyr = (tmp.groupby('chain_class2')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count+pyr)
  count1 = tmp.groupby('for_profit')[case].sum().map('{:,}'.format)
  pyr1 = (tmp.groupby('for_profit')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count1+pyr1)

**Table 3**

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))



**Figure 2**

In [48]:
d['FIRST_SE'] = pd.to_datetime(d['FIRST_SE'], format='%m/%d/%Y')
fig2 = pd.DataFrame()

for event, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  print('-'*50, event, '-'*50)
  for exposure in ['chain_class2', 'for_profit']:
    print('\n', exposure)
    fig2 = pd.DataFrame()
    for year in range(2001, 2016, 2):
        num = d[(d['FIRST_SE']>=str(year)+'/01/01') & (d['FIRST_SE']<=str(year+1)+'/12/31')].groupby(exposure)[event].sum()
        pyr = d[(d['FIRST_SE']>=str(year)+'/01/01') & (d['FIRST_SE']<=str(year+1)+'/12/31')].groupby(exposure)[time].sum()/12
        rate = (num/pyr*100).rename(str(year)+'-'+"{0:0>2}".format(year+1-2000))
        fig2 = pd.concat([fig2, rate], 1)
    print(fig2.applymap('{:.2f}'.format))

-------------------------------------------------- wl --------------------------------------------------

 chain_class2
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
1    2.67    2.59    2.67    2.56    2.74    3.02    2.90    3.75
2    3.03    2.97    3.17    2.94    2.88    3.29    3.31    3.94
3    3.06    2.91    2.99    2.70    2.90    3.14    3.09    3.75
4    3.92    3.26    3.76    2.96    2.87    2.94    2.92    3.64
5   15.68   17.94   18.65   19.26   17.81   18.32   16.79   24.15
6    6.28    6.45    6.90    6.03    6.46    6.41    5.90    7.82

 for_profit
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
0    9.93   11.00   11.73   11.58   11.12   11.27   10.21   14.09
1    2.95    2.83    2.99    2.76    2.83    3.14    3.09    3.81
-------------------------------------------------- livingd --------------------------------------------------

 chain_class2
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
1    1.09    0.9

In [43]:
fig2.head(n=6)

,2001-2002,2003-2004,2005-2006,2007-2008,2009-2010,2011-2012,2013-2014,2015-2016
0,1.604619,1.631538,1.421149,1.029222,0.813118,0.808248,0.770415,1.218847
1,0.623987,0.567080,0.518337,0.438927,0.356887,0.354080,0.335175,0.478963


**Supplemental Table 3 (ideal cohort)**

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model'+str(i))
      cph.fit(PH_data_ideal.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


**Supplemental Table 4**

In [0]:
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for r, region in enumerate(['Northeast', 'South', 'Midwest', 'West']):
    print('\n', region)
    for exposure in ['chain', 'for_profit']:
      crude = '|'.join([exposure, time, status])
      model1 = crude + '|sex_new|age_cat|race_new'
      model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                        'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
      model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #network_us_region_dfr'
      cph.fit(PH_data[d['network_us_region_dfr']==r].filter(regex=model3), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))

Supplemental Table 5 & 6

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['profit_txc', 'profit_hosp']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                      'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn|network_us_region_dfr' #network_us_region_dfr'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.2)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))